# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
# Import API key
from api_keys import g_key 


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:

Weather_csv = pd.read_csv("../starter_code/Weather.csv")
Weather_csv.head()

,Unnamed: 0,Location,Temperature [°C],Humidity [%],Cloudiness[%],WIndspeed [kph],Latitude,Longitude
0,0,Hampshire,8.23,71,75,2.06,42.3334,-72.5829
1,1,Savigny-sur-Orge,6.73,71,0,2.06,48.6768,2.3484
2,2,Arrondissement de Nevers,0.00,88,4,1.34,46.9167,3.3333
3,3,28 de Noviembre,6.30,70,83,11.55,-51.6500,-72.3000
4,4,Laguna Woods,18.41,44,0,3.22,33.6103,-117.7253


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps with API key.
# api_keys.py
# g_key

gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
Weather_csv = Weather_csv.dropna()
Humidity = Weather_csv["Humidity [%]"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = Weather_csv[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=max(Weather_csv["Humidity [%]"]),
                                 point_radius =3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# PErfect Weather conditions, %70 -%100 humidity, less than 2kph wind, 25-35 degrees,20% cloudiness Tropical Paradise!!! 

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:

Holiday_Weather = Weather_csv

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# Filtering on temperature conditions
Holiday_Weather = Holiday_Weather.loc[(Holiday_Weather["Temperature [°C]"] < 35) & (Holiday_Weather["Temperature [°C]"] > 15)]

 #less than 2 kph.
Holiday_Weather = Holiday_Weather.loc[Holiday_Weather["WIndspeed [kph]"] < 10]

#cloudiness.
Holiday_Weather = Holiday_Weather.loc[Holiday_Weather["Cloudiness[%]"] <= 50]

#Humidity 

Holiday_Weather = Holiday_Weather.loc[(Holiday_Weather["Humidity [%]"] < 100) & (Holiday_Weather["Humidity [%]"] > 40)]
# Too restrictive a list, cannot find hotels with filters. increased cloudiness to 50%o,                

# Holiday_Weather=Holiday_Weather.dropna()

Holiday_Weather


,Unnamed: 0,Location,Temperature [°C],Humidity [%],Cloudiness[%],WIndspeed [kph],Latitude,Longitude
4,4,Laguna Woods,18.41,44,0,3.22,33.6103,-117.7253
7,7,Ali Sabih,24.00,60,1,2.06,11.1558,42.7125
11,11,Oranjestad,26.53,74,20,8.75,12.5240,-70.0270
17,17,Sakété,28.00,83,40,4.63,6.7362,2.6587
40,40,Irvine,19.07,60,0,1.34,33.6695,-117.8231
...,...,...,...,...,...,...,...,...
438,438,Calverton,15.82,55,1,3.09,39.0576,-76.9358
441,441,Barra Bonita,22.73,69,20,3.09,-22.4947,-48.5581
444,444,Hybla Valley,15.67,63,1,2.57,38.7476,-77.0830
447,447,Qingping,20.07,68,0,1.28,29.0282,110.2281


In [64]:
from pprint import pprint
hotel_df = Holiday_Weather
lat = 39.0576
lng = -76.9358

# set parameters for hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location" :f"{lat},{lng}"
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response = requests.get(base_url, params=params).json()
pprint(response)

{'html_attributions': [],
 'next_page_token': 'ATtYBwKVMLVHNyrZs9lWyrl6UVymOl57A1TmP2h5MJRJH99hSN0SxiicbGFPQ8J6Ike02xa3l6GHrZaYJ55yL3NDuG64BhXvMKS5mIp-Qs7U7qKG9tVD8dnjkcK8z9_riCwr023DSWAEc02cOJm1lcEJHbtJ9E7mkn-itIY5yKvC_LB7A07VDMkZwlAjy7ThOOSvx5_owo5u2vQCPyQz_RQOnqNeGjTieJ0y6LaFldEfPRcZs1bejr1BAC_7Gv0Yg3vI9KqUMjhlYDB6ZcBt9zGUNg85Y_XmJvw-XbfQb-GOMVp-jRC2iMMKx_9OMXSImBlLoU6_ykkdSY6dYA5gF2VfqOdtF3gx38Z3_JmSGDfW70Lbf6IZc5czNO73nC6ADNyjbb0O56fPHDt-IrfpjzqH-zPZ9d3gM_Y-trZrbVTlywVB39ZqGuEpgWIP',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 39.0614032, 'lng': -76.9604532},
                           'viewport': {'northeast': {'lat': 39.0625727302915,
                                                      'lng': -76.95858091970848},
                                        'southwest': {'lat': 39.0598747697085,
                                                      'lng': -76.96127888029149}}},
              'icon': 'https://maps.gstatic.com/mapfil

In [69]:
# for index, row in hotel_df.iterrows():
#     # extracting location information
#     lat = row["Latitude"]
#     lng = row["Longitude"]
    

-117.7253
42.7125
-70.027
2.6587
-117.8231
-92.2833
-48.9111
-84.0633
-84.2725
-80.2833
100.0833
-84.5925
-77.2922
-82.2056
17.0832
-158.1547
-50.5547
-119.0376
-96.7836
140.7667
-76.7936
-122.3344
-117.7271
-14.0131
120.7409
-77.15
153.0407
-90.7344
-158.0092
149.1281
-117.3793
-83.5085
-97.8333
109.0172
-80.2298
114.1577
-87.2068
-88.6503
-99.1517
100.0
-72.35
23.7833
-79.7833
31.5756
-98.3856
-118.4912
-9.471
-84.4777
174.85
121.0078
-109.9124
-89.5554
122.2047
101.85700000000001
36.8167
-82.7412
-118.1853
-68.4681
-97.6003
120.1253
120.214
-118.0903
-66.0074
-82.5842
-77.0725
-98.1561
120.6898
14.3667
-47.7
-99.5833
-77.3997
16.340999999999998
-76.9358
-48.5581
-77.083
110.2281
-80.4333


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [71]:
from pprint import pprint
hotel_df = Holiday_Weather

# set parameters for hotel
params = {
    "radius": 50000,
    "types": "lodging",
    "key": g_key
#     "location" :f"{lat},{lng}"
}

# base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# response = requests.get(base_url, params=params).json()
# pprint(response)

for index, row in hotel_df.iterrows():
    # extracting location information
    lat = row["Latitude"]
    lng = row["Longitude"]

    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#     response = requests.get(base_url, params=params).json()
    
    # assemble url and make API request
   
    try:
        print(f"Retrieving Results for Index {index}: {row['Location']}.")
        response = requests.get(base_url, params=params).json()
    
    except (KeyError, IndexError):
        print("Nothing Here!!")
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Nothing Here!!")
        
#     print("------------")

hotel_df
# pprint(response)

Retrieving Results for Index 4: Laguna Woods.
Closest hotel is Laguna Hills Lodge.
Retrieving Results for Index 7: Ali Sabih.
Closest hotel is Arta Maison des Randonneurs.
Retrieving Results for Index 11: Oranjestad.
Closest hotel is Holiday Inn Resort Aruba-Beach Resort & Casino.
Retrieving Results for Index 17: Sakété.
Closest hotel is Comfort zone.
Retrieving Results for Index 40: Irvine.
Closest hotel is Residence Inn by Marriott Irvine John Wayne Airport/Orange County.
Retrieving Results for Index 47: Tapachula.
Closest hotel is Hotel Casa Mexicana.
Retrieving Results for Index 58: Arealva.
Closest hotel is Quality Suites Garden Bauru.
Retrieving Results for Index 60: Beavercreek.
Closest hotel is Marriott at the University of Dayton.
Retrieving Results for Index 70: Candler-McAfee.
Closest hotel is Emory Conference Center Hotel.
Retrieving Results for Index 74: Las Tablas.
Closest hotel is Gran Hotel Azuero.
Retrieving Results for Index 77: Phatthalung.
Closest hotel is Ao Thai R

,Unnamed: 0,Location,Temperature [°C],Humidity [%],Cloudiness[%],WIndspeed [kph],Latitude,Longitude,Hotel Name
4,4,Laguna Woods,18.41,44,0,3.22,33.6103,-117.7253,Laguna Hills Lodge
7,7,Ali Sabih,24.00,60,1,2.06,11.1558,42.7125,Arta Maison des Randonneurs
11,11,Oranjestad,26.53,74,20,8.75,12.5240,-70.0270,Holiday Inn Resort Aruba-Beach Resort & Casino
17,17,Sakété,28.00,83,40,4.63,6.7362,2.6587,Comfort zone
40,40,Irvine,19.07,60,0,1.34,33.6695,-117.8231,Residence Inn by Marriott Irvine John Wayne Ai...
...,...,...,...,...,...,...,...,...,...
438,438,Calverton,15.82,55,1,3.09,39.0576,-76.9358,Kimpton Hotel Monaco Washington DC
441,441,Barra Bonita,22.73,69,20,3.09,-22.4947,-48.5581,Hotel Jau
444,444,Hybla Valley,15.67,63,1,2.57,38.7476,-77.0830,Kimpton Hotel Monaco Washington DC
447,447,Qingping,20.07,68,0,1.28,29.0282,110.2281,Minnan International Hotel


In [72]:
hotel_df.head()

,Unnamed: 0,Location,Temperature [°C],Humidity [%],Cloudiness[%],WIndspeed [kph],Latitude,Longitude,Hotel Name
4,4,Laguna Woods,18.41,44,0,3.22,33.6103,-117.7253,Laguna Hills Lodge
7,7,Ali Sabih,24.00,60,1,2.06,11.1558,42.7125,Arta Maison des Randonneurs
11,11,Oranjestad,26.53,74,20,8.75,12.5240,-70.0270,Holiday Inn Resort Aruba-Beach Resort & Casino
17,17,Sakété,28.00,83,40,4.63,6.7362,2.6587,Comfort zone
40,40,Irvine,19.07,60,0,1.34,33.6695,-117.8231,Residence Inn by Marriott Irvine John Wayne Ai...


In [ ]:
# locations = hotel_df[["Latitude", "Longitude"]]
# markers = gmaps.marker_layer(locations)

# # Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """

# # Store the DataFrame Row
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# # Create hotel symbol layer
# hotel_layer = gmaps.symbol_layer(
#     locations, fill_color='rgba(0, 150, 0, 0.4)',
#     stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
#     info_box_content=hotel_info
# )
# # Add the layer to the map
# fig.add_layer(markers)
# fig.add_layer(hotel_layer)
# fig

# <dt>Country</dt><dd>{Country}</dd>

In [80]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Location}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]



In [82]:
markers = gmaps.marker_layer(locations)

hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))